# Importing libaries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

# Importing first databases

Radars.csv contains all cars, trucks, motorcycles and buses that comes thru São Paulo's radar system 


In [42]:

df = pd.read_csv(r"D:\\Users\\guilh\\Documents\\GitHub\\Dados_CET\\Marco_2018_nAg\\2_nAg.csv", usecols= ["Data", "Numero Agrupado", "Faixa", "Registro", "Especie", "Classe", "Velocidade"], index_col= "Data")


MemoryError: Unable to allocate 256. KiB for an array with shape (32768,) and data type int64

## Reading the columns

* **Radar** is the number of identification of a street section
* **Lane** goes from 1 to 6 in most radars, low lane number are closer to the center of the freeway, high lane numbers are "local" lanes, to the right
* **Register** represents each vehicle
* **Types** are: motorcycle = 0, car = 1, bus = 2 ou truck = 3
* **Classes** are: *light* (motorcycle and car) = 0 ou *heavy* (bus and truck)  = 1
* **Speeds** are in kilometer per hour


In [44]:
# Preprocessing

# turns speed from dm/s to km/h
df["Speed [km/h]"] = df["Speed [km/h]"] * 0.36

df.index.names = ["Date"]


####################################################### Stop Point ########################################
df["Radar_Lane"] = str(zip(df["Radar"],df["Lane"]))

# renaming columns to english
df.columns = ["Radar", "Lane", "Register", "Type", "Class", "Speed [km/h]", "Radar_Lane"]

df.head()

,Radar,Lane,Register,Type,Class,Speed [km/h],Radar_Lane
Date,,,,,,,
2018-03-01 23:56:00,10588,2,32526,1.0,0.0,13.3488,<zip object at 0x00000200B52CD288>
2018-03-01 23:56:00,10588,2,32527,0.0,0.0,8.9424,<zip object at 0x00000200B52CD288>
2018-03-01 23:56:00,10588,3,32528,1.0,0.0,13.7376,<zip object at 0x00000200B52CD288>
2018-03-01 23:57:00,10588,2,32529,0.0,0.0,5.4432,<zip object at 0x00000200B52CD288>
2018-03-01 23:57:00,10588,1,32530,1.0,0.0,6.8688,<zip object at 0x00000200B52CD288>


In [ ]:
df.info()

In [ ]:
lane_types = pd.read_excel(r"D:\Users\guilh\Documents\[POLI]_6_Semestre\IC\2021\codigos olimpio\\Faixa Tipo.xlsx", usecols = ["Num_agrupado","faixa", "Num_fx","tipo"])


In [ ]:
lane_types.head()